In [ ]:
from dotenv import load_dotenv

load_dotenv()

## Few shots prompt example

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain import LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import AIMessage, HumanMessage, SystemMessage

# Initialize the chat model
chat = ChatOpenAI(model_name="gpt-3.5-turbo-16k", temperature=0)

In [ ]:
# Define the system message (role of the AI)
template = "You are the greatest joke teller ever, telling jokes based on the user's company role that you have to guess from these options: project manager, data scientist, developer, designer, researcher, operations, executive team, business developer."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)

# Define example conversations between human and AI
example_conversations = [
    {
        "human": "I organize and oversee all aspects of our team's projects",
        "ai": "Why don't project managers play hide and seek? Because good luck hiding when the schedule's always behind!",
    },
    {
        "human": "I analyze and interpret complex digital data to help our company make decisions",
        "ai": "Why did the data scientist get kicked out of the supermarket? They kept putting the ketchup in the basket, but it kept regressing to the mean!",
    },
    {
        "human": "I write and test code for our software applications",
        "ai": "Why did the developer go broke? Because he used up all his cache!",
    },
    {
        "human": "I ensure that our company's daily activities run smoothly",
        "ai": "Why don't operations people play poker? Too many suits!",
    },
]


# Define the templates for human and AI messages
example_humans = [
    HumanMessagePromptTemplate.from_template(conversation["human"])
    for conversation in example_conversations
]
example_ais = [
    AIMessagePromptTemplate.from_template(conversation["ai"])
    for conversation in example_conversations
]
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

# Build the chat prompt
chat_prompt = ChatPromptTemplate.from_messages(
    [system_message_prompt]
    + [msg for pair in zip(example_humans, example_ais) for msg in pair]
    + [human_message_prompt]
)

In [ ]:
for i, message in enumerate(chat_prompt.messages):
    print(f"Message {i+1}:")
    print(f"Type: {type(message).__name__}")
    if hasattr(message.prompt, "template"):
        print(f"Template: {message.prompt.template}")
    print()

In [ ]:
chain = LLMChain(llm=chat, prompt=chat_prompt)

chain.run("I ask a lot of questions to the users")

In [ ]:
chain.run("Who become a detective?")

## Memory

Given the {past_conversation} answer a question

### Conversation buffer memory
This memory allows for storing of messages and then extracts the messages in a variable.

In [ ]:
# Define the system message (role of the AI)
template = """You are the greatest joke teller ever, telling jokes based on the user's company role that you have to guess from these options: project manager, data scientist, developer, designer, researcher, operations, executive team, business developer. You can also remember the names and actions of the people because you have memory. If I ask you about 'whow' or about someone's name, you don't have to tell a joke, just answer based on our previous conversations.
{chat_history}
"""
system_message_prompt = SystemMessagePromptTemplate.from_template(template)

# Define example conversations between human and AI
example_conversations = [
    {
        "human": "I organize and oversee all aspects of our team's projects",
        "ai": "Why don't project managers play hide and seek? Because good luck hiding when the schedule's always behind!",
    },
    {
        "human": "I analyze and interpret complex digital data to help our company make decisions",
        "ai": "Why did the data scientist get kicked out of the supermarket? They kept putting the ketchup in the basket, but it kept regressing to the mean!",
    },
    {
        "human": "I write and test code for our software applications",
        "ai": "Why did the developer go broke? Because he used up all his cache!",
    },
    {
        "human": "I ensure that our company's daily activities run smoothly",
        "ai": "Why don't operations people play poker? Too many suits!",
    },
]


# Define the templates for human and AI messages
example_humans = [
    HumanMessagePromptTemplate.from_template(conversation["human"])
    for conversation in example_conversations
]
example_ais = [
    AIMessagePromptTemplate.from_template(conversation["ai"])
    for conversation in example_conversations
]
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

# Build the chat prompt
chat_prompt = ChatPromptTemplate.from_messages(
    [system_message_prompt]
    + [msg for pair in zip(example_humans, example_ais) for msg in pair]
    + [human_message_prompt]
)

In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [ ]:
chain = LLMChain(
    llm=chat,
    prompt=chat_prompt,
    verbose=True,
    memory=memory,
)

chain.run("I ask a lot of questions to the users")

In [ ]:
memory.load_memory_variables({})

In [ ]:
chain.run("The researcher's name is Kanika")

In [ ]:
chain.run("What is the researcher's name?")

In [ ]:
memory.load_memory_variables({})

### Conversation buffer window memory
Keeps a list of the interactions of the conversation over time. It only uses the last K interactions. This can be useful for keeping a sliding window of the most recent interactions, so the buffer does not get too large.

```
memory = ConversationBufferWindowMemory(k=2)
```

### Conversation token buffer memory
keeps a buffer of recent interactions in memory, and uses token length rather than number of interactions to determine when to flush interactions.

```
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=10)
```

### Conversation summary memory
This type of memory creates a summary of the conversation over time.

In [ ]:
from langchain.memory import ConversationSummaryMemory
from langchain.llms import OpenAI

In [ ]:
from langchain.chains import ConversationChain

memory = ConversationSummaryMemory(llm=OpenAI())
llm = ChatOpenAI(temperature=0)
chain = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True,
)

In [ ]:
# Define the system message (role of the AI)
template = "You are the greatest joke teller ever, telling jokes based on the user's company role that you have to guess from these options: project manager, data scientist, developer, designer, researcher, operations, executive team, business developer."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)

# Define example conversations between human and AI
example_conversations = [
    {
        "human": "I organize and oversee all aspects of our team's projects",
        "ai": "Why don't project managers play hide and seek? Because good luck hiding when the schedule's always behind!",
    },
    {
        "human": "I analyze and interpret complex digital data to help our company make decisions",
        "ai": "Why did the data scientist get kicked out of the supermarket? They kept putting the ketchup in the basket, but it kept regressing to the mean!",
    },
    {
        "human": "I write and test code for our software applications",
        "ai": "Why did the developer go broke? Because he used up all his cache!",
    },
    {
        "human": "I ensure that our company's daily activities run smoothly",
        "ai": "Why don't operations people play poker? Too many suits!",
    },
]


# Define the templates for human and AI messages
example_humans = [
    HumanMessagePromptTemplate.from_template(conversation["human"])
    for conversation in example_conversations
]
example_ais = [
    AIMessagePromptTemplate.from_template(conversation["ai"])
    for conversation in example_conversations
]
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

# Build the chat prompt
chat_prompt = ChatPromptTemplate.from_messages(
    [system_message_prompt]
    + [msg for pair in zip(example_humans, example_ais) for msg in pair]
    + [human_message_prompt]
)

In [ ]:
prompt = chat_prompt.format(text="I plan what people have to do")
chain.predict(input=prompt)

In [ ]:
prompt = chat_prompt.format(text="what do I do?")
chain.predict(input=prompt)

In [ ]:
prompt = chat_prompt.format(text="My name is Clara")
chain.predict(input=prompt)

In [ ]:
memory.load_memory_variables({})

In [ ]:
prompt = chat_prompt.format(text="What is my name?")
chain.predict(input=prompt)

In [ ]:
memory.load_memory_variables({})

### Conversation summary buffer memory
It keeps a buffer of recent interactions in memory, but rather than just completely flushing old interactions it compiles them into a summary and uses both.

````
memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=10)
````